# 리뷰 특성 벡터화 후 클러스터링
## Import

In [8]:
import pandas as pd
from pandas import DataFrame

import numpy as np

import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

from sklearn.decomposition import PCA

from tqdm import tqdm

### Data Load

In [16]:
Raw_Data = pd.read_csv("./Original_Data_UTF8.CSV", encoding='utf-8')

Data_Set = pd.DataFrame(Raw_Data["Review"])

Data_Set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10658 entries, 0 to 10657
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  10657 non-null  object
dtypes: object(1)
memory usage: 83.4+ KB


In [15]:
Data_Set['length'] = ''

for index in tqdm(Data_Set['Review']):
   # print(index + "\n")
    Data_Set['length'] = len(index)
    
# Data_Set.head()

 96%|█████████████████████████████████████████████████████████████████████▊   | 10193/10658 [00:00<00:00, 11078.59it/s]


TypeError: object of type 'float' has no len()